In [40]:
import pandas as pd

# Load the dataset
file_path = '../data/raw/land_use_data.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataset
df[:5]

,Period,Subdivision,Cropland_pct,Grassland_pct,Forest_pct,Wetland_pct,Cropland_area,Grassland_area,Forest_area,Wetland_area,Biodiversity_Score
0,Year-2000,Subdivision_1,48.423590,22.447908,17.976951,NaN,4842.358998,2244.790823,1797.695130,NaN,0.482540
1,Year-2001,Subdivision_1,51.335362,22.963853,17.747514,NaN,5133.536248,2296.385289,1774.751423,NaN,0.463808
2,Year-2002,Subdivision_1,51.479157,22.624337,17.708999,NaN,5147.915682,2262.433700,1770.899942,NaN,0.463585
3,Year-2003,Subdivision_1,51.392751,20.299837,17.129947,NaN,5139.275142,2029.983661,1712.994723,NaN,0.468174
4,Year-2004,Subdivision_1,51.475015,22.063842,17.344408,NaN,5147.501494,2206.384233,1734.440775,NaN,0.464367


In [41]:
# Check the data types of the columns
df.dtypes

Period                 object
Subdivision            object
Cropland_pct          float64
Grassland_pct         float64
Forest_pct            float64
Wetland_pct           float64
Cropland_area         float64
Grassland_area        float64
Forest_area           float64
Wetland_area          float64
Biodiversity_Score    float64
dtype: object

In [43]:
# Clean up Year data and transform it into datetime format
df['Year'] = df['Period'].str[-4:].astype(int)
df['Year'] = pd.to_datetime(df['Year'], format='%Y')

# Remove unnecessary subdivision name prefix
df['Subdivision'] = df['Subdivision'].str.split('_').str[-1]

df[:5]

,Period,Subdivision,Cropland_pct,Grassland_pct,Forest_pct,Wetland_pct,Cropland_area,Grassland_area,Forest_area,Wetland_area,Biodiversity_Score,Year
0,Year-2000,1,48.423590,22.447908,17.976951,NaN,4842.358998,2244.790823,1797.695130,NaN,0.482540,2000-01-01
1,Year-2001,1,51.335362,22.963853,17.747514,NaN,5133.536248,2296.385289,1774.751423,NaN,0.463808,2001-01-01
2,Year-2002,1,51.479157,22.624337,17.708999,NaN,5147.915682,2262.433700,1770.899942,NaN,0.463585,2002-01-01
3,Year-2003,1,51.392751,20.299837,17.129947,NaN,5139.275142,2029.983661,1712.994723,NaN,0.468174,2003-01-01
4,Year-2004,1,51.475015,22.063842,17.344408,NaN,5147.501494,2206.384233,1734.440775,NaN,0.464367,2004-01-01


In [44]:
# Calculate the number of missing values in each column
missing_data = df.isnull().sum()
print(missing_data)

Period                  0
Subdivision             0
Cropland_pct           22
Grassland_pct           0
Forest_pct              0
Wetland_pct           210
Cropland_area          21
Grassland_area          0
Forest_area             0
Wetland_area          210
Biodiversity_Score     10
Year                    0
dtype: int64


In [45]:
# The missing values summary shows us that there is one instance of missing data in Cropland_pct column (year 2015 and subdivision 5), while the data in the corresponding area column is present

# Upon inspection of the Cropland_pct, Grassland_pct, Forest_pct and Wetland_pct columns, it appears that they are all related to the area columns by a factor of 100
# Therefore, I can populate missing values in the Cropland_pct column (Year-2015, Subdivision_5) by dividing the Cropland_area by 100
df.loc[(df['Year'] == '2015') & (df['Subdivision'] == '5'), 'Cropland_pct'] = df.loc[(df['Year'] == '2015') & (df['Subdivision'] == '5'), 'Cropland_area'] / 100

In [46]:
# I noticed that the Wetland_pct and Wetland_area column has missing values

# We have been instructed that the land cover percentages across types (Cropland, Grassland, Forest, Wetland) sum to 100% for each year and subdivision.
# Therefore, I can calculate the missing data in the Wetland_pct and Wetland_area by subtracting the sum of the other three land cover types from 100 and multiplying by 100 to get the total area, respectively.
df['Wetland_pct'] = 100 - df['Cropland_pct'] - df['Grassland_pct'] - df['Forest_pct']
df['Wetland_area'] = df['Wetland_pct'] * 100

# Unfortunately, this only applies in the circumstance where Cropland, Grassland and Forest data are complete. 
# We can see that there are missing values in the Cropland_pct and Cropland_area columns for Subdivision 2 (all years).
# Show the dataset for Subdivision 2
df[df['Subdivision'] == '2']

,Period,Subdivision,Cropland_pct,Grassland_pct,Forest_pct,Wetland_pct,Cropland_area,Grassland_area,Forest_area,Wetland_area,Biodiversity_Score,Year
21,Year-2000,2,NaN,27.326952,12.440651,NaN,NaN,2732.695220,1244.065105,NaN,0.450371,2000-01-01
22,Year-2001,2,NaN,25.334916,13.600897,NaN,NaN,2533.491647,1360.089709,NaN,0.457383,2001-01-01
23,Year-2002,2,NaN,27.190195,11.680888,NaN,NaN,2719.019464,1168.088756,NaN,0.445810,2002-01-01
24,Year-2003,2,NaN,25.123455,12.481971,NaN,NaN,2512.345495,1248.197089,NaN,0.469028,2003-01-01
25,Year-2004,2,NaN,26.275104,12.598570,NaN,NaN,2627.510438,1259.856967,NaN,0.457913,2004-01-01
26,Year-2005,2,NaN,27.119793,10.591567,NaN,NaN,2711.979300,1059.156664,NaN,0.443667,2005-01-01
27,Year-2006,2,NaN,26.431439,13.001260,NaN,NaN,2643.143940,1300.125990,NaN,0.446319,2006-01-01
28,Year-2007,2,NaN,24.765394,9.432851,NaN,NaN,2476.539369,943.285057,NaN,0.434063,2007-01-01
29,Year-2008,2,NaN,25.388442,8.805831,NaN,NaN,2538.844156,880.583065,NaN,0.432063,2008-01-01
30,Year-2009,2,NaN,26.455474,9.451707,NaN,NaN,2645.547445,945.170687,NaN,0.435850,2009-01-01


In [47]:
# We can see that there is missing Biodiversity_Score data for the year 2010 (all subdivisions).
missing_biodiversity = df[df['Biodiversity_Score'].isnull()]
print(missing_biodiversity)


        Period Subdivision  Cropland_pct  Grassland_pct  Forest_pct  \
10   Year-2010           1     52.404273      22.145197   15.044479   
31   Year-2010           2           NaN      25.846784    7.047486   
52   Year-2010           3     51.744078      23.799736   12.844597   
73   Year-2010           4     49.721480      29.206553   12.516540   
94   Year-2010           5     51.407197      20.758634   12.924339   
115  Year-2010           6     51.869196      24.632852    9.245753   
136  Year-2010           7     54.751715      27.438484    7.610269   
157  Year-2010           8     54.297419      24.501838    8.147547   
178  Year-2010           9     50.786360      25.090949    9.280672   
199  Year-2010          10     50.169985      23.550702   12.664914   

     Wetland_pct  Cropland_area  Grassland_area  Forest_area  Wetland_area  \
10     10.406052    5240.427269     2214.519657  1504.447879   1040.605195   
31           NaN            NaN     2584.678421   704.748593  